# Analysis of the DSAIR model

> A number of replications and extensions of The Anh et al.'s analysis of their DSAIR model.

In [ ]:
#| default_exp analysis_dsair

In [ ]:
#| hide
#| export
from gh_pages_example.conditions import *
from gh_pages_example.methods import *
from gh_pages_example.payoffs import *
from gh_pages_example.types import *
from gh_pages_example.utils import *

import typing

import fastcore.test
import matplotlib as mpl
import matplotlib.pyplot as plt
from nbdev.showdoc import *
import nptyping
import numpy as np
import pandas
import seaborn as sns 

This notebook contains a number of analyses of different versions of the DSAIR model.

Each analysis involves a number of steps:

1. Create parameter space
2. Run the model
3. Process the results
4. Visualise results and explain what we observe

## Baseline DSAIR model

### Create parameter space

In [ ]:
#| export
def fig1_data(b:float=4,
              c:float=1,
              B:float=10**4,
              W:float=100,
              β:float=0.01,
              Z:int=100,
              S:list[str]=["AS", "AU"],
              collective:float=1) -> dict: # A dictionary containing the items in `ModelTypeDSAIR`
    """Initialise baseline DSAIR models which vary `s` and `p`. By default,
    we create models for replicating Figure 1 of The Anh et al. 2021."""
    namesofvalues=['s','b','c','p','B','W','β','collective']
    matchingvalues = np.array([[s,b,c,p,B,W,β,collective]
                               for s in np.arange(1,5.1,0.1)
                               for p in np.arange(0,1.02,0.02)])
    models = {k:v for k, v in zip(namesofvalues, matchingvalues.T)}
    models = {**models,
              'Z':Z, # Z should be a scalar
              'strategy_set':S # S should be a list of strings
             }
    return models

In [ ]:
show_doc(fig1_data)

---

[source](https://github.com/PaoloBova/gh-pages-example/blob/main/gh_pages_example/analysis_dsair.py#L25){target="_blank" style="float:right; font-size:smaller"}

### fig1_data

>      fig1_data (b:float=4, c:float=1, B:float=10000, W:float=100,
>                 β:float=0.01, Z:int=100, S:list[str]=['AS', 'AU'],
>                 collective:float=1)

Initialise baseline DSAIR models which vary `s` and `p`. By default,
we create models for replicating Figure 1 of The Anh et al. 2021.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| b | float | 4 |  |
| c | float | 1 |  |
| B | float | 10000 |  |
| W | float | 100 |  |
| β | float | 0.01 |  |
| Z | int | 100 |  |
| S | list | ['AS', 'AU'] |  |
| collective | float | 1 |  |
| **Returns** | **dict** |  | **A dictionary containing the items in `ModelTypeDSAIR`** |

In [ ]:
#| export
models = fig1_data()

In [ ]:
#| export
b=4; c=1; B=10**4; W=100; β=0.1; Z=100; S=["AS", "AU"]; collective=1
models = fig1_data(b=b, c=c, B=B, W=W, β=β, Z=Z, S=S, collective=collective)

### Run the model

In [ ]:
#| export
results = thread_macro(models,
                       payoffs_sr,
                       payoffs_lr,
                       threshold_society_prefers_safety_dsair,
                       threshold_risk_dominant_safety_dsair,
                       markov_chain,
                      )

In [ ]:
#| hide
{k:v.shape for k,v in results.items() if isinstance(v, np.ndarray)}

### Process the results

Now that we have collected some results, we need to process them so that we can display what we want to.

The general approach I follow is to flatten the `results` dictionary and convert it into a [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) dataframe.

Here, we also compute the risk of an AI related disaster, $p_{risk} = 1 - p$

In [ ]:
#| hide
#| export

flat_results = {k:results[k] for k in ['s','b','c','p','B','W','β',
                                       'threshold_society_prefers_safety',
                                       'threshold_risk_dominant_safety']}
flat_results['pr'] = np.round(1 - flat_results['p'], 2)
flat_results['s'] = np.round(flat_results['s'], 2)
for i, strategy in enumerate(["AS", "AU"]):
    flat_results[strategy + "_frequency"] = results['ergodic'][:,i]
    
df = pandas.DataFrame(flat_results)

In [ ]:
#| echo:false
df

### Visualise results and explain what we observe

I am using the [Matplotlib](https://matplotlib.org/stable/index.html) library to visualize our data.

#| hide

:::{.callout-note}
Note that there are five types of callouts, including: 
`note`, `tip`, `warning`, `caution`, and `important`.
:::

In [ ]:
#| hide
sns.set_theme(style='darkgrid',palette='deep' ,font='sans-serif', font_scale=1.4)
plt.rcParams["axes.grid"] = False

I first replicate the Figure 1 from The Anh Han et al. 2021

The figure describes how the the frequency of Always Unsafe (**AU**) varies with both the speed advantage given to those who play **AU**, $s$, and the risk that such firms cause an AI disaster, $p_{risk}$. We have also plotted two lines, the lower line shows the boundary where society is indifferent between the two strategies. A greater risk or a slower speed advantage from this boundary implies society prefers players to play Always Safe (**AS**). The higher line shows the threshold for which **AU** is risk dominant over **AS**. For this baseline model, risk dominance implies that the strategy will be selected for by evolution (which is why the line follows the boundary where players switch from **AU** to **AS**). As with the lower line, any higher risk or lower speed implies that **AS** will instead by risk dominant over **AU**.

These lines therefore split the heatmap into 3 regions. 
(i) Society prefers **AS** and **AS** is selected by social learning.
(ii) Society prefers **AS** but **AU** is selected by social learning
(iii) Society prefers **AU** and **AU** is selected by social learning

In region (i) companies will be alligned with Society's preference for safety. In region (iii), society is willing to accept the risks as they anticipate greater benefits from innovation. In region (ii), we see a dilemma where all players are choosing to play **AU**, even though society prefers them to play **AS**. We can refer to this region as the *Dilemma zone*.

In [ ]:
#| echo:false
table = df.pivot_table(index='pr', columns='s', values='AU_frequency')

heatmap, ax = plt.subplots()
im = ax.imshow(table.values,
               cmap='inferno',
               extent=[table.columns.min(),
                       table.columns.max(),
                       table.index.min(),
                       table.index.max()],
               interpolation='nearest',
               origin='lower',
               aspect='auto')
ax.set(xlabel='Speed avantage, s',
       ylabel='Risk of an AI disaster, pr')

cbar = heatmap.colorbar(im)
cbar.ax.set_ylabel('AU Frequency')

# Add threshold boundaries to convey dilemma region
plt.plot(table.columns, df[df.pr==1]['threshold_society_prefers_safety'])
plt.plot(table.columns, df[df.pr==1]['threshold_risk_dominant_safety']);

:::{.callout-caution}
Note that this model is illustrative only: at best here, society refers to the collection of all firms.

We could instead explicitly model society's preferences over safety and innovation, distinct from the companies. Such a model will still have the 3 regions we are currently discussing, though the negative externalities of an AI disaster will likely lead to a greater *dillemma zone*. 

Another insight that such an extension would communicate is that companies may have incentive to work together to make sure their preferences are weighted more highly than the rest of society. It would be interesting to see whether we can observe this in pracitse, for examle in the European AI Act. The main challenge this task presents us is how to determine whether companies are working together to have their voices heard or whether they each already have strong enough incentives to uniltaterally influence policy.
:::

I have also plotted a cross-section of the above heatmap for speed advantage, $s=1.5$. This plot shows how players are distributed between the 2 available strategies, $AU$ and $AS$. Here, the blue area represents the proportion of players who follow $AU$ for each level of risk, whereas the red area tells us the proportion who play $AS$. I also mark the 3 regions discussed above.

We will often make use of this cross-section plot in more complex models when we want to show the relative frequencies of more than 2 strategies. 

In [ ]:
#| echo:false
dfplot2 = df[df.s==1.5]
x = dfplot2.pr.values
y1 = dfplot2.AU_frequency.values
y2 = dfplot2.AS_frequency.values

fig, ax = plt.subplots()
ax.stackplot(x,
             [y1, y2],
             labels=['AU', 'AS'],
             alpha=0.8)
ax.legend(loc='upper left')
ax.set_title('Strategy distribution')
ax.set_xlabel('Risk of an AI disaster, pr')
ax.set_ylabel('Proportion')

# Add threshold boundaries to convey dilemma region
plt.vlines([dfplot2[dfplot2.pr==1]['threshold_society_prefers_safety'],
            dfplot2[dfplot2.pr==1]['threshold_risk_dominant_safety']],
           0,
           0.995,
           colors=['C2', 'C3'],
           linewidth=3,
           label='pr*');

#| hide

### Explain the patterns observed

#| hide

# References

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()